# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [ ]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup

import time
from tqdm import tqdm

In [2]:

def flatten(fatlist:list) -> list:
    
    flat_list = []

    for xs in fatlist:
        for x in xs:
            flat_list.append(x)

    flat_list = list(filter(None, flat_list))

    return flat_list

In [21]:

url = "https://en.wikipedia.org/wiki/South_Gippsland"
response = requests.get(url)
filename = "Wikipedia_SouthGippsland.html"

with open(filename, mode='wb') as file:
    file.write(response.content)
    
with open(filename) as fp:
    bs = BeautifulSoup(fp, 'html.parser')


town_names = []

regex = re.compile('navbox-list-with-group navbox-list navbox-*')

for link in bs.find_all("td", {"class" : regex}):
    town_names.append(link.text.split('\n'))

towns = flatten(town_names)

print(towns)
len(towns)

['Boisdale', 'Briagolong', 'Coongulla', 'Cowwarr', 'Fernbank', 'Glengarry', 'Gormandale', 'Heyfield', 'Hollands Landing', 'Licola', 'Loch Sport', 'Longford', 'Maffra', 'Munro', 'Newry', 'Rosedale', 'Sale', 'Seaspray', 'Stratford', 'Tinamba', 'Toongabbie', 'Woodside', 'Woodside Beach', 'Anglers Rest', 'Bairnsdale', 'Bellbird Creek', 'Bemm River', 'Benambra', 'Bendoc', 'Buchan', 'Buchan South', 'Bruthen', 'Cabbage Tree Creek', 'Cann River', 'Cape Conran', 'Cassilis', 'Club Terrace', 'Dargo', 'Deddick', 'Dinner Plain', 'Eagle Point', 'Ensay', 'Gelantipy', 'Genoa', 'Johnsonville', 'Lakes Entrance', 'Lake Tyers Beach', 'Lindenow', 'Mallacoota', 'Marlo', 'Metung', 'Mossiface', 'Mount Hotham', 'Nicholson', 'Nowa Nowa', 'Nungurner', 'Omeo', 'Orbost', 'Paynesville', 'Raymond Island', 'Swan Reach', 'Swifts Creek', 'Tambo Crossing', 'Tambo Upper', 'Twin Rivers', 'W Tree', 'Woodglen', 'Wulgulmerang', 'Boolarra', 'Churchill', 'Erica', 'Glengarry', 'Hazelwood North', 'Moe', 'Morwell', 'Newborough', 

165

In [22]:
#https://stackoverflow.com/questions/4576077/how-can-i-split-a-text-into-sentences
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'


from nltk.tokenize import sent_tokenize

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead 
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences



In [23]:
text = []
df_data = pd.DataFrame(text,columns=['text','town'])

#uncomment for quick run
towns = ['Briagolong', 'Foster']

for town in towns:
    print(town)
    # Get the Wikipedia page for the town
    params = {
        "action": "query", 
        "prop": "extracts",
        "exlimit": 1,
        "titles": f"{town}, Victoria",
        "explaintext": 1,
        "formatversion": 2,
        "format": "json"
    }
    

    resp = requests.get("https://en.wikipedia.org/w/api.php", params=params)
    response_dict = resp.json()
        
    try:
        temp_list = response_dict["query"]["pages"][0]["extract"].split("\n")
        df_response = pd.DataFrame(temp_list)
        df_response.columns = ["text"]
        df_response.insert(0,'town',town)
        df_response.dropna(inplace = True)
        
        #clean up the response
        df_response = df_response[(df_response["text"].str.len() > 0) & (~df_response["text"].str.startswith("==") & (~df_response["text"].str.startswith("http://")))]
        df_response = df_response[~df_response.text.str.contains("\t\t")]
        df_response = df_response.map(lambda x: x.replace('"', ''))
        
        #split into sentences and flatten
        result = flatten([split_into_sentences(x) for x in df_response['text']])
        
        result_list = []
        for x in result:
            result_list.append(x)
            
        df = pd.DataFrame(result_list)

        df.columns = ["text"]
        df['text'] = town + ": " + df['text'].astype(str)
        
        #df.insert(0,'town',town)
        
        #store each towns data in seperate file in case we need to use it later for something else
        df.to_csv(f"data/{town}.csv",mode='a',header=False, index=False)
        #store all the data in one file
        df.to_csv(f"data/town_data.csv",mode='a',header=False, index=False)

    except:
        print(f"{town} - No text found")
    finally:
        pass
    for i in tqdm(range(200)):
        time.sleep(0.02)


Briagolong


100%|██████████| 200/200 [00:04<00:00, 49.07it/s]


Foster


100%|██████████| 200/200 [00:04<00:00, 49.12it/s]


In [24]:
df = pd.read_csv(f"data/town_data.csv",header=0,names=['text'])
pd.options.display.max_colwidth = 200
df

,text
0,"Briagolong: At the 2016 census, Briagolong had a population of 1,081."
1,Briagolong: Briagolong Post Office opened on 1 May 1871.
2,"Briagolong: The town's principal industry has been timber, and it supplied red gum paving blocks for the streets of Melbourne, and stringybark for the flooring in Australia House in London."
3,Briagolong: A railway branch line from Maffra opened in 1889 and was closed in 1952.
4,Briagolong: The town's local cricket team (the Saints) were crowned Sale-Maffra Cricket Association 2018/19 season premiers.
5,"Briagolong: Briagolong has a pub, which serves meals and drinks."
6,Briagolong: It was established on July 14th 1874.
7,"Briagolong: Briagolong was the home town of Private Jake Kovco, Australia's first military casualty in the Iraq War."
8,"Briagolong: Private Kovco's funeral was held in Briagolong on 2 May 2006, and was attended by Prime Minister John Howard, Defence Minister Brendan Nelson, Defence Force Chief Angus Houston and Arm..."
9,"Briagolong: In conjunction with neighbouring township Boisdale, Briagolong has an Australian Rules football team in the East Gippsland Football League."


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [ ]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "XXXXX"

In [26]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,"Briagolong: At the 2016 census, Briagolong had a population of 1,081.","[-0.011834477074444294, 0.008756508119404316, -0.003429737174883485, -0.004711177200078964, 0.008511527441442013, -0.002443530596792698, -0.00693485327064991, 0.007431096863001585, -0.030176661908..."
1,Briagolong: Briagolong Post Office opened on 1 May 1871.,"[-0.02111160382628441, -0.0033099243883043528, -0.03092259168624878, 0.015730969607830048, 0.005281111225485802, -0.0027577353175729513, 0.00018841047130990773, 0.0055443644523620605, -0.008635981..."
2,"Briagolong: The town's principal industry has been timber, and it supplied red gum paving blocks for the streets of Melbourne, and stringybark for the flooring in Australia House in London.","[-0.00514441030099988, -0.001068274606950581, -0.011377125047147274, -0.014541888609528542, 0.0017342771170660853, 0.028522932901978493, -0.02591901272535324, -0.0040794736705720425, -0.0006714272..."
3,Briagolong: A railway branch line from Maffra opened in 1889 and was closed in 1952.,"[-0.026271408423781395, -0.0052699740044772625, -0.004746899474412203, 0.007061503361910582, -0.0012807147577404976, 0.00963764451444149, 0.0061559309251606464, -0.0040570953860878944, -0.01515607..."
4,Briagolong: The town's local cricket team (the Saints) were crowned Sale-Maffra Cricket Association 2018/19 season premiers.,"[-0.014033523388206959, -0.00178688729647547, -0.008004209026694298, 0.002306768437847495, -0.019003456458449364, 0.027726998552680016, -0.00337432324886322, -0.014949037693440914, -0.012091325595..."
5,"Briagolong: Briagolong has a pub, which serves meals and drinks.","[-0.00835717748850584, -0.009816974401473999, -0.01318279467523098, -0.006973876617848873, 0.0034582524094730616, -0.011257647536695004, 0.0028160056099295616, 0.005581013858318329, 2.858636071323..."
6,Briagolong: It was established on July 14th 1874.,"[-0.017045456916093826, 0.00488213961943984, -0.014536489732563496, -0.005189293995499611, 0.006634536664932966, -0.013553595170378685, 0.011827063746750355, 0.010365655645728111, -0.0197613481432..."
7,"Briagolong: Briagolong was the home town of Private Jake Kovco, Australia's first military casualty in the Iraq War.","[-0.02458622120320797, -0.0005992972292006016, 0.0060815466567873955, -0.010336354374885559, -0.005252688191831112, 0.025639358907938004, -0.007937539368867874, -0.0095172468572855, 0.000786196731..."
8,"Briagolong: Private Kovco's funeral was held in Briagolong on 2 May 2006, and was attended by Prime Minister John Howard, Defence Minister Brendan Nelson, Defence Force Chief Angus Houston and Arm...","[-0.019704455509781837, -0.00541774183511734, -0.0027973640244454145, -0.008161026984453201, 0.001873103203251958, 0.035711869597435, 0.00578482449054718, -0.011674529872834682, -0.018262347206473..."
9,"Briagolong: In conjunction with neighbouring township Boisdale, Briagolong has an Australian Rules football team in the East Gippsland Football League.","[-0.027026865631341934, 0.002653734292834997, 0.0064700874499976635, -0.00042148964712396264, -0.018382906913757324, 0.02970636449754238, -0.0033268295228481293, -2.8280466722208075e-05, -0.008405..."


In [27]:
df.to_csv("embeddings.csv")

#df = pd.read_csv("embeddings.csv")


In [28]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [29]:
get_rows_sorted_by_relevance("Where did Australia's first military casualty in the Iraq War come from?", df)

,text,embeddings,distances
7,"Briagolong: Briagolong was the home town of Private Jake Kovco, Australia's first military casualty in the Iraq War.","[-0.02458622120320797, -0.0005992972292006016, 0.0060815466567873955, -0.010336354374885559, -0.005252688191831112, 0.025639358907938004, -0.007937539368867874, -0.0095172468572855, 0.000786196731...",0.128122
8,"Briagolong: Private Kovco's funeral was held in Briagolong on 2 May 2006, and was attended by Prime Minister John Howard, Defence Minister Brendan Nelson, Defence Force Chief Angus Houston and Arm...","[-0.019704455509781837, -0.00541774183511734, -0.0027973640244454145, -0.008161026984453201, 0.001873103203251958, 0.035711869597435, 0.00578482449054718, -0.011674529872834682, -0.018262347206473...",0.207740
13,"Briagolong: Irving Mosquito, Australian rules footballer","[-0.03418387845158577, 0.0009378112154081464, -0.0032988500315696, -0.00024518478312529624, -0.0034639574587345123, 0.02373586967587471, 0.0014380872016772628, -0.01201983354985714, -0.02221687883...",0.222780
12,"Briagolong: Len Maxwell, Australian rules footballer","[-0.019886502996087074, 0.012201571837067604, 0.0008867228170856833, -0.012294561602175236, -0.02207840234041214, 0.02850797213613987, 0.016020789742469788, -0.027206117287278175, -0.0200857669115...",0.228683
2,"Briagolong: The town's principal industry has been timber, and it supplied red gum paving blocks for the streets of Melbourne, and stringybark for the flooring in Australia House in London.","[-0.00514441030099988, -0.001068274606950581, -0.011377125047147274, -0.014541888609528542, 0.0017342771170660853, 0.028522932901978493, -0.02591901272535324, -0.0040794736705720425, -0.0006714272...",0.237082
6,Briagolong: It was established on July 14th 1874.,"[-0.017045456916093826, 0.00488213961943984, -0.014536489732563496, -0.005189293995499611, 0.006634536664932966, -0.013553595170378685, 0.011827063746750355, 0.010365655645728111, -0.0197613481432...",0.239960
23,"Foster: When the gold ran out, Foster became a service centre of the burgeoning South Gippsland dairy industry.","[-0.007390913553535938, -0.00047513016033917665, -0.0011589546920731664, -0.011132563464343548, -0.03030274622142315, 0.024812353774905205, -0.00773406308144331, -0.002723749727010727, -0.01213561...",0.241803
26,"Foster: Because of the scenic beauty of the surrounding area and its proximity to Wilsons Promontory and mountain areas inland, Foster is a popular destination for international visitors as well a...","[0.011441851034760475, -0.017961198464035988, -0.009152160957455635, -0.004077891353517771, -0.0292710792273283, 0.029746172949671745, -0.021643178537487984, 0.0045925769954919815, -0.007324368227...",0.246128
35,"Foster: Foster railway station, Victoria","[-0.009217575192451477, -0.00920424610376358, -0.0012138474266976118, 0.0031308431643992662, -0.024393584579229355, 0.023860391229391098, -0.009197580628097057, -0.011896871030330658, -0.008204508...",0.250043
36,"Foster: visitmelbourne.com: Foster, Victoria","[-0.008438082411885262, -0.01647530496120453, -0.01451109629124403, -0.0210183747112751, -0.0199494156986475, 0.03409973904490471, -0.010308757424354553, -0.008458125405013561, -0.0269243624061346...",0.250406


In [30]:
get_rows_sorted_by_relevance("When was the post office in Foster, Victoria opened?", df)

,text,embeddings,distances
19,Foster: The post office opened on 20 February 1871 as Stockyard Creek and was renamed Foster in 1879 when the township was established.,"[-0.0045533571392297745, -0.007672535255551338, -0.013351368717849255, 0.0007006092346273363, -0.030793042853474617, 0.0072995200753211975, -0.017081519588828087, 0.011852877214550972, 0.003201177...",0.110534
35,"Foster: Foster railway station, Victoria","[-0.009217575192451477, -0.00920424610376358, -0.0012138474266976118, 0.0031308431643992662, -0.024393584579229355, 0.023860391229391098, -0.009197580628097057, -0.011896871030330658, -0.008204508...",0.115319
1,Briagolong: Briagolong Post Office opened on 1 May 1871.,"[-0.02111160382628441, -0.0033099243883043528, -0.03092259168624878, 0.015730969607830048, 0.005281111225485802, -0.0027577353175729513, 0.00018841047130990773, 0.0055443644523620605, -0.008635981...",0.142443
36,"Foster: visitmelbourne.com: Foster, Victoria","[-0.008438082411885262, -0.01647530496120453, -0.01451109629124403, -0.0210183747112751, -0.0199494156986475, 0.03409973904490471, -0.010308757424354553, -0.008458125405013561, -0.0269243624061346...",0.143275
22,Foster: The story of Foster is in a publication available at the Foster & District Historical Society Inc. The railway was extended to Foster in 1892.,"[-0.007895582355558872, -0.006276656407862902, 0.0026904696132987738, -0.01618926227092743, -0.030733058229088783, 0.002048538764938712, -0.009527778252959251, 0.016255611553788185, -0.01445090677...",0.155469
24,Foster: Today it is a thriving town that links Wilsons Promontory with the rest of Victoria.,"[-0.004018546547740698, -0.003910114988684654, -0.013215531595051289, -0.00012085623166058213, -0.01846625655889511, 0.024512149393558502, -0.01373468991369009, 0.012518939562141895, 0.00566473789...",0.155679
23,"Foster: When the gold ran out, Foster became a service centre of the burgeoning South Gippsland dairy industry.","[-0.007390913553535938, -0.00047513016033917665, -0.0011589546920731664, -0.011132563464343548, -0.03030274622142315, 0.024812353774905205, -0.00773406308144331, -0.002723749727010727, -0.01213561...",0.166475
14,"Foster: Foster is a dairying and grazing town 174 kilometres (108 mi) south-east of Melbourne on the South Gippsland Highway in Victoria, Australia.","[-0.0020520116668194532, 0.007126315031200647, -0.01105988398194313, -0.0034320384729653597, -0.030340924859046936, 0.04122379794716835, -0.022355781868100166, -0.0009497929131612182, -0.019379381...",0.170570
32,Foster: The Great Southern Rail Trail links Foster to Leongatha in the west and Yarram in the east.,"[0.010823359712958336, 0.005887141916900873, 0.006715897936373949, -0.00584421819075942, -0.0213033314794302, 0.02813149057328701, -0.013880843296647072, -0.009113018400967121, -0.0172883197665214...",0.186400
26,"Foster: Because of the scenic beauty of the surrounding area and its proximity to Wilsons Promontory and mountain areas inland, Foster is a popular destination for international visitors as well a...","[0.011441851034760475, -0.017961198464035988, -0.009152160957455635, -0.004077891353517771, -0.0292710792273283, 0.029746172949671745, -0.021643178537487984, 0.0045925769954919815, -0.007324368227...",0.187481


In [31]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [32]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
        

### Question 1

In [33]:
brigolong_prompt = """
Question: "Where did Australia's first military casualty in the Iraq War come from?"
Answer:
"""
initial_brigolong_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=brigolong_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_brigolong_answer)

The first Australian military casualty in the Iraq War was a military interpreter named Jacob Kovco, who was originally from Melbourne, Victoria.


In [34]:
print(create_prompt("Where did Australia's first military casualty in the Iraq War come from?", df, 200))

custom_briagolong_answer = answer_question("Where did Australia's first military casualty in the Iraq War come from?", df)
print(custom_briagolong_answer)


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Briagolong: Briagolong was the home town of Private Jake Kovco, Australia's first military casualty in the Iraq War.

###

Briagolong: Private Kovco's funeral was held in Briagolong on 2 May 2006, and was attended by Prime Minister John Howard, Defence Minister Brendan Nelson, Defence Force Chief Angus Houston and Army Chief Peter Leahy.

###

Briagolong: Irving Mosquito, Australian rules footballer

###

Briagolong: Len Maxwell, Australian rules footballer

---

Question: Where did Australia's first military casualty in the Iraq War come from?
Answer:
Briagolong


### Question 2

In [35]:
postoffice_prompt = """
Question: "When was the post office in Foster, Victoria opened?"
Answer:
"""
initial_postoffice_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=postoffice_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_postoffice_answer)

I'm sorry, I am an AI and do not have information about specific historical events. Can I assist you with anything else?


In [36]:
print(create_prompt("When was the post office in Foster opened?", df, 100))

custom_postoffice_answer = answer_question("When was the post office in Foster opened?", df)
print(custom_postoffice_answer)


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Foster: The post office opened on 20 February 1871 as Stockyard Creek and was renamed Foster in 1879 when the township was established.

---

Question: When was the post office in Foster opened?
Answer:
20 February 1871


## Rationale for Dataset

As we can see the LLM is unable to specifically answer the questions and by augmenting the LLM with additional data it can answer our questions correctly